In [4]:
import jax
import jax.numpy as jnp
import numpy as np
# import jax.example_libraries.optimizers as jopt

import flax
# import haiku

import optax

# from jax_metrics.metrics import Accuracy

In [3]:
RANDOMKEY = jax.random.PRNGKey(412)

x = jax.random.normal(RANDOMKEY, shape = (1024, 10), dtype = jnp.float32)
weights = jax.random.uniform(RANDOMKEY, shape = (10,), dtype = jnp.float32)
y = jnp.matmul(x,weights)
y = jnp.where( y > 0.5, 1.0, 0.0)

In [11]:
def init_random_params(scale, layer_sizes, random_key : jax.random.PRNGKey):
  return [(scale * jax.random.normal(key = random_key, shape = (m,n)), scale * jax.random.normal(key = random_key, shape = (n,))) for m, n in zip(layer_sizes[:-1], layer_sizes[1:])]

@jax.jit
def predict(params, inputs):
  activations = inputs
  for w, b in params[:-1]:
    outputs = jnp.dot(activations, w) + b
    activations = jnp.tanh(outputs)

  final_w, final_b = params[-1]
  logits = jnp.dot(activations, final_w) + final_b
  return logits - jax.scipy.special.logsumexp(logits, axis=1, keepdims=True)

@jax.jit
def loss(params, inputs, label):
  preds = predict(params, inputs)
  return -jnp.mean(jnp.sum(preds*label, axis = 1))

@jax.jit
def update(params, inputs, label, lr):
  grads = jax.grad(loss)(params, inputs, label)
  return [(w - lr * gradw, b - lr * gradb) for (w,b),(gradw, gradb) in zip(params, grads)] 

In [12]:
import time

params = init_random_params(1,[10,32,32,16,2],RANDOMKEY)


BATCHSIZE = 32

def data_stream():
    for i in range(0, y.shape[0], BATCHSIZE):
      yield x[i:i+BATCHSIZE,], y[i:i+BATCHSIZE]

t = [time.time()]
for e in range(50):
    for inputs, label in data_stream():
        params = update(params, inputs, jax.nn.one_hot(label,2), 1e-2)

t.append(time.time())

In [5]:
pred = predict(params, inputs)
pred = jnp.argmax(pred, axis = 1)

pred == label

DeviceArray([ True, False,  True,  True,  True,  True,  True,  True,
              True,  True,  True,  True,  True,  True,  True,  True,
              True,  True,  True,  True,  True,  True,  True,  True,
              True,  True,  True,  True,  True,  True,  True,  True],            dtype=bool, weak_type=True)

In [6]:
def build_optax_update_fn(optimizer):
    @jax.jit
    def update(params, opt_state, inputs, label):
        lossvalue, grads = jax.value_and_grad(loss)(params, inputs, label)
        updates, opt_state = optimizer.update(grads, opt_state)
        params = optax.apply_updates(params, updates)
        return params, opt_state, lossvalue
    return update

params = init_random_params(1,[10,32,32,16,2],RANDOMKEY)
optimizer = optax.sgd(1e-2)
optax_update_fn = build_optax_update_fn(optimizer)
opt_state = optimizer.init(params)

lr_scheduler = optax.exponential_decay(1e-1, 4, -1, 5)

for e in range(100):
    for inputs, label in data_stream():
        params, opt_state, lossvalue = optax_update_fn(params, opt_state, inputs, jax.nn.one_hot(label,2))

In [7]:
pred = predict(params, inputs)
pred = jnp.argmax(pred, axis = 1)
label == pred

DeviceArray([ True, False,  True,  True,  True,  True,  True,  True,
              True,  True,  True,  True,  True,  True,  True,  True,
              True,  True,  True,  True,  True,  True,  True,  True,
              True,  True,  True,  True,  True,  True,  True,  True],            dtype=bool, weak_type=True)

In [1]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

key = random.PRNGKey(0)
x = random.normal(key, (10,))
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
%timeit jnp.dot(x, x.T).block_until_ready()  # runs on the GPU

6.11 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
import numpy as np

x = np.random.normal(size = (size,size))
%timeit np.dot(x,x.T)

133 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

1.79 ms ± 564 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

231 µs ± 20 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * np.where(x > 0, x, alpha * np.exp(x) - alpha)

x = np.random.normal(size = (1000000,))
%timeit selu(x)

7.14 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
